In [2]:
%pip install -r requirements.txt

  Using cached baukit-0.0.1-py3-none-any.whl
  Using cached neuron_explainer-0.0.1-py3-none-any.whl
  Using cached transformer_lens-0.0.0-py3-none-any.whl
  Using cached aiohttp-3.8.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached anyio-3.7.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached asttokens-2.4.0-py2.py3-none-any.whl.metadata (4.9 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached autobahn-23.6.1-py2.py3-none-any.whl
  Using cached Automat-22.10.0-py2.py3-none-any.whl (26 kB)
  Using cached black-23.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
  Using cached blobfile-2.0.2-py3-none-any.whl (74 kB)
  Using cached blosc2-2.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.9 MB)
  Using cached boostedblob-0.13.2-py3-none-any.whl (58 kB)
  Using cach

In [26]:
import torch 
import argparse
from utils import dotdict
from activation_dataset import setup_token_data
import wandb
import json
from datetime import datetime
from tqdm import tqdm
from einops import rearrange
import matplotlib.pyplot as plt

cfg = dotdict()
# models: "EleutherAI/pythia-6.9b", "lomahony/eleuther-pythia6.9b-hh-sft", "usvsnsp/pythia-6.9b-ppo", "Dahoas/gptj-rm-static", "reciprocate/dahoas-gptj-rm-static"
# cfg.model_name="lomahony/eleuther-pythia6.9b-hh-sft"
# "EleutherAI/pythia-70m", "lomahony/pythia-70m-helpful-sft", "lomahony/eleuther-pythia70m-hh-sft"
cfg.model_name="EleutherAI/pythia-70m-deduped"
cfg.layers=[0,]
cfg.setting="residual"
# cfg.tensor_name="gpt_neox.layers.{layer}" or "transformer.h.{layer}"
cfg.tensor_name="gpt_neox.layers.{layer}"
original_l1_alpha = 8e-4
cfg.l1_alpha=original_l1_alpha
cfg.l1_alphas=[8e-5, 1e-4, 2e-4, 4e-4, 8e-4, 1e-3, 2e-3, 4e-3, 8e-3]
cfg.sparsity=None
cfg.num_epochs=10
cfg.model_batch_size=8
cfg.lr=1e-3
cfg.kl=False
cfg.reconstruction=False
#cfg.dataset_name="NeelNanda/pile-10k"
cfg.dataset_name="Elriggs/openwebtext-100k"
cfg.device="cuda:0"
cfg.ratio = 4
cfg.seed = 0
# cfg.device="cpu"

In [27]:
tensor_names = [cfg.tensor_name.format(layer=layer) for layer in cfg.layers]

In [28]:
# Load in the model
from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, GPTJForSequenceClassification
model = AutoModelForCausalLM.from_pretrained(cfg.model_name)
model = model.to(cfg.device)
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)

In [29]:
# Download the dataset
# TODO iteratively grab dataset?
cfg.max_length = 256
token_loader = setup_token_data(cfg, tokenizer, model, seed=cfg.seed)
num_tokens = cfg.max_length*cfg.model_batch_size*len(token_loader)
print(f"Number of tokens: {num_tokens}")

Found cached dataset parquet (/root/.cache/huggingface/datasets/Elriggs___parquet/Elriggs--openwebtext-100k-79076ecafee8a6d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Map (num_proc=8):   0%|          | 0/100000 [00:00<?, ? examples/s]

Number of tokens: 112750592


In [30]:
# Run 1 datapoint on model to get the activation size
from baukit import Trace

text = "1"
tokens = tokenizer(text, return_tensors="pt").input_ids.to(cfg.device)
# Your activation name will be different. In the next cells, we will show you how to find it.
with torch.no_grad():
    with Trace(model, tensor_names[0]) as ret:
        _ = model(tokens)
        representation = ret.output
        # check if instance tuple
        if(isinstance(representation, tuple)):
            representation = representation[0]
        activation_size = representation.shape[-1]
print(f"Activation size: {activation_size}")

Activation size: 512


In [6]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install --upgrade functorch

Note: you may need to restart the kernel to use updated packages.


In [31]:
# Initialize New autoencoder
from autoencoders.learned_dict import AnthropicSAE
from torch import nn
autoencoders = []
optimizers = []
for layer in cfg.layers:
    l1_variants = []
    l1_optimizers = []
    for l1 in cfg.l1_alphas:
        params = dict()
        n_dict_components = activation_size*cfg.ratio
        params["encoder"] = torch.empty((n_dict_components, activation_size), device=cfg.device)
        nn.init.xavier_uniform_(params["encoder"])

        params["decoder"] = torch.empty((n_dict_components, activation_size), device=cfg.device)
        nn.init.xavier_uniform_(params["decoder"])

        params["encoder_bias"] = torch.empty((n_dict_components,), device=cfg.device)
        nn.init.zeros_(params["encoder_bias"])

        params["shift_bias"] = torch.empty((activation_size,), device=cfg.device)
        nn.init.zeros_(params["shift_bias"])

        autoencoder = AnthropicSAE(  # TiedSAE, UntiedSAE, AnthropicSAE
            # n_feats = n_dict_components, 
            # activation_size=activation_size,
            encoder=params["encoder"],
            encoder_bias=params["encoder_bias"],
            decoder=params["decoder"],
            shift_bias=params["shift_bias"],
        )
        autoencoder.to_device(cfg.device)
        autoencoder.set_grad()
        l1_variants.append(autoencoder)

        optimizer = torch.optim.Adam(
            [
                autoencoder.encoder, 
                autoencoder.encoder_bias,
                autoencoder.decoder,
                autoencoder.shift_bias,
            ], lr=cfg.lr)
        l1_optimizers.append(optimizer)
        
    autoencoders.append(l1_variants)
    optimizers.append(l1_optimizers)

In [32]:
# Set target sparsity to 10% of activation_size if not set
if cfg.sparsity is None:
    cfg.sparsity = int(activation_size*0.05)
    print(f"Target sparsity: {cfg.sparsity}")

target_lower_sparsity = cfg.sparsity * 0.9
target_upper_sparsity = cfg.sparsity * 1.1
adjustment_factor = 0.1  # You can set this to whatever you like

Target sparsity: 25


In [33]:
original_bias = autoencoder.encoder_bias.clone().detach()
# Wandb setup
secrets = json.load(open("secrets.json"))
wandb.login(key=secrets["wandb_key"])
start_time = datetime.now().strftime("%Y%m%d-%H%M%S")
wandb_run_name = f"{cfg.model_name}_{start_time[4:]}_{cfg.sparsity}"  # trim year
print(f"wandb_run_name: {wandb_run_name}")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb_run_name: EleutherAI/pythia-70m-deduped_1210-055534_25


In [34]:
wandb.init(project="sparse coding", config=dict(cfg), name=wandb_run_name)

In [ ]:
# time_since_activation = torch.zeros(autoencoder.encoder.shape[0])
total_activations = torch.zeros(autoencoder.encoder.shape[0])
max_num_tokens = 100_000_000
save_every = 30_000
num_saved_so_far = 0
# Freeze model parameters 
model.eval()
model.requires_grad_(False)
model.to(cfg.device)
# last_encoder = autoencoder.encoder.clone().detach()
for i, batch in enumerate(tqdm(token_loader,total=int(max_num_tokens/(cfg.max_length*cfg.model_batch_size)))):
    tokens = batch["input_ids"].to(cfg.device)
    with torch.no_grad(): # As long as not doing KL divergence, don't need gradients for model
        
        #print(tensor_names)
        
        with Trace(model, tensor_names[0]) as ret:
            _ = model(tokens)
            representation = ret.output
            if(isinstance(representation, tuple)):
                representation = representation[0]
    #print(f"representation is: {representation}")
    print(f"representation shape is: {representation.shape}")
    layer_activations = rearrange(representation, "b seq d_model -> (b seq) d_model")
    # activation_saver.save_batch(layer_activations.clone().cpu().detach())
    for layer in range(len(cfg.layers)):
        for l1_alpha in range(len(cfg.l1_alphas)):
            autoencoder = autoencoders[layer][l1_alpha]
            optimizer = optimizers[layer][l1_alpha]
            
            c = autoencoder.encode(layer_activations)
            x_hat = autoencoder.decode(c)
            
            reconstruction_loss = (x_hat - layer_activations).pow(2).mean()
            l1_loss = torch.norm(c, 1, dim=-1).mean()
            total_loss = reconstruction_loss + cfg.l1_alpha*l1_loss

            time_since_activation += 1
            time_since_activation = time_since_activation * (c.sum(dim=0).cpu()==0)
            # total_activations += c.sum(dim=0).cpu()
            if ((i) % 100 == 0): # Check here so first check is model w/o change
                # self_similarity = torch.cosine_similarity(c, last_encoder, dim=-1).mean().cpu().item()
                # Above is wrong, should be similarity between encoder and last encoder
                self_similarity = torch.cosine_similarity(autoencoder.encoder, last_encoder, dim=-1).mean().cpu().item()
                last_encoder = autoencoder.encoder.clone().detach()

                num_tokens_so_far = i*cfg.max_length*cfg.model_batch_size
                with torch.no_grad():
                    sparsity = (c != 0).float().mean(dim=0).sum().cpu().item()
                    # Count number of dead_features are zero
                    num_dead_features = (time_since_activation >= min(i, 200)).sum().item()
                print(f"Sparsity: {sparsity:.1f} | Dead Features: {num_dead_features} | Total Loss: {total_loss:.2f} | Reconstruction Loss: {reconstruction_loss:.2f} | L1 Loss: {cfg.l1_alpha*l1_loss:.2f} | l1_alpha: {cfg.l1_alpha:.2e} | Tokens: {num_tokens_so_far} | Self Similarity: {self_similarity:.2f}")
                wandb.log({
                    'Sparsity': sparsity,
                    'Dead Features': num_dead_features,
                    'Total Loss': total_loss.item(),
                    'Reconstruction Loss': reconstruction_loss.item(),
                    'L1 Loss': (cfg.l1_alpha*l1_loss).item(),
                    'l1_alpha': cfg.l1_alpha,
                    'Tokens': num_tokens_so_far,
                    'Self Similarity': self_similarity
                })
                
                dead_features = torch.zeros(autoencoder.encoder.shape[0])
                
                if(num_tokens_so_far > max_num_tokens):
                    print(f"Reached max number of tokens: {max_num_tokens}")
                    break
                
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
    
    # resample_period = 10000
    # if (i % resample_period == 0):
    #     # RESAMPLING
    #     with torch.no_grad():
    #         # Count number of dead_features are zero
    #         num_dead_features = (total_activations == 0).sum().item()
    #         print(f"Dead Features: {num_dead_features}")
            
    #     if num_dead_features > 0:
    #         print("Resampling!")
    #         # hyperparams:
    #         max_resample_tokens = 1000 # the number of token activations that we consider for inserting into the dictionary
    #         # compute loss of model on random subset of inputs
    #         resample_loader = setup_token_data(cfg, tokenizer, model, seed=i)
    #         num_resample_data = 0

    #         resample_activations = torch.empty(0, activation_size)
    #         resample_losses = torch.empty(0)

    #         for resample_batch in resample_loader:
    #             resample_tokens = resample_batch["input_ids"].to(cfg.device)
    #             with torch.no_grad(): # As long as not doing KL divergence, don't need gradients for model
    #                 with Trace(model, tensor_names[0]) as ret:
    #                     _ = model(resample_tokens)
    #                     representation = ret.output
    #                     if(isinstance(representation, tuple)):
    #                         representation = representation[0]
    #             layer_activations = rearrange(representation, "b seq d_model -> (b seq) d_model")
    #             resample_activations = torch.cat((resample_activations, layer_activations.detach().cpu()), dim=0)

    #             c = autoencoder.encode(layer_activations)
    #             x_hat = autoencoder.decode(c)
                
    #             reconstruction_loss = (x_hat - layer_activations).pow(2).mean(dim=-1)
    #             l1_loss = torch.norm(c, 1, dim=-1)
    #             temp_loss = reconstruction_loss + cfg.l1_alpha*l1_loss
                
    #             resample_losses = torch.cat((resample_losses, temp_loss.detach().cpu()), dim=0)
                
    #             num_resample_data +=layer_activations.shape[0]
    #             if num_resample_data > max_resample_tokens:
    #                 break

                
    #         # sample num_dead_features vectors of input activations
    #         probabilities = resample_losses**2
    #         probabilities /= probabilities.sum()
    #         sampled_indices = torch.multinomial(probabilities, num_dead_features, replacement=True)
    #         new_vectors = resample_activations[sampled_indices]

    #         # calculate average encoder norm of alive neurons
    #         alive_neurons = list((total_activations!=0))
    #         modified_columns = total_activations==0
    #         avg_norm = autoencoder.encoder.data[alive_neurons].norm(dim=-1).mean()

    #         # replace dictionary and encoder weights with vectors
    #         new_vectors = new_vectors / new_vectors.norm(dim=1, keepdim=True)
            
    #         params_to_modify = [autoencoder.encoder, autoencoder.encoder_bias]

    #         current_weights = autoencoder.encoder.data
    #         current_weights[modified_columns] = (new_vectors.to(cfg.device) * avg_norm * 0.02)
    #         autoencoder.encoder.data = current_weights

    #         current_weights = autoencoder.encoder_bias.data
    #         current_weights[modified_columns] = 0
    #         autoencoder.encoder_bias.data = current_weights
            
    #         if hasattr(autoencoder, 'decoder'):
    #             current_weights = autoencoder.decoder.data
    #             current_weights[modified_columns] = new_vectors.to(cfg.device)
    #             autoencoder.decoder.data = current_weights
    #             params_to_modify += [autoencoder.decoder]

    #         for param_group in optimizer.param_groups:
    #             for param in param_group['params']:
    #                 if any(param is d_ for d_ in params_to_modify):
    #                     # Extract the corresponding rows from m and v
    #                     m = optimizer.state[param]['exp_avg']
    #                     v = optimizer.state[param]['exp_avg_sq']
                        
    #                     # Update the m and v values for the modified columns
    #                     m[modified_columns] = 0  # Reset moving average for modified columns
    #                     v[modified_columns] = 0  # Reset squared moving average for modified columns
        
    #     total_activations = torch.zeros(autoencoder.encoder.shape[0])
    
    
    

    # if ((i+2) % save_every ==0): # save periodically but before big changes
    #     model_save_name = cfg.model_name.split("/")[-1]
    #     save_name = f"{model_save_name}_sp{cfg.sparsity}_r{cfg.ratio}_{tensor_names[0]}_ckpt{num_saved_so_far}"  # trim year

    #     # Make directory traiend_models if it doesn't exist
    #     import os
    #     if not os.path.exists("trained_models"):
    #         os.makedirs("trained_models")
    #     # Save model
    #     torch.save(autoencoder, f"trained_models/{save_name}.pt")
        
    #     num_saved_so_far += 1

    # # Running sparsity check
    # num_tokens_so_far = i*cfg.max_length*cfg.model_batch_size
    # if(num_tokens_so_far > 200000):
    #     if(i % 100 == 0):
    #         with torch.no_grad():
    #             sparsity = (c != 0).float().mean(dim=0).sum().cpu().item()
    #         if sparsity > target_upper_sparsity:
    #             cfg.l1_alpha *= (1 + adjustment_factor)
    #         elif sparsity < target_lower_sparsity:
    #             cfg.l1_alpha *= (1 - adjustment_factor)
    #         # print(f"Sparsity: {sparsity:.1f} | l1_alpha: {cfg.l1_alpha:.2e}")

  0%|          | 1/48828 [00:00<2:19:28,  5.83it/s]

representation shape is: torch.Size([8, 256, 512])
Sparsity: 1031.1 | Dead Features: 2048 | Total Loss: 0.32 | Reconstruction Loss: 0.19 | L1 Loss: 0.13 | l1_alpha: 8.00e-04 | Tokens: 0 | Self Similarity: -0.00
Sparsity: 1030.8 | Dead Features: 2048 | Total Loss: 0.32 | Reconstruction Loss: 0.19 | L1 Loss: 0.13 | l1_alpha: 8.00e-04 | Tokens: 0 | Self Similarity: 0.00
Sparsity: 1004.0 | Dead Features: 2048 | Total Loss: 0.31 | Reconstruction Loss: 0.18 | L1 Loss: 0.13 | l1_alpha: 8.00e-04 | Tokens: 0 | Self Similarity: -0.00
Sparsity: 1023.5 | Dead Features: 2048 | Total Loss: 0.31 | Reconstruction Loss: 0.18 | L1 Loss: 0.13 | l1_alpha: 8.00e-04 | Tokens: 0 | Self Similarity: -0.00
Sparsity: 1026.6 | Dead Features: 2048 | Total Loss: 0.32 | Reconstruction Loss: 0.19 | L1 Loss: 0.13 | l1_alpha: 8.00e-04 | Tokens: 0 | Self Similarity: 0.00
Sparsity: 1008.9 | Dead Features: 2048 | Total Loss: 0.31 | Reconstruction Loss: 0.18 | L1 Loss: 0.13 | l1_alpha: 8.00e-04 | Tokens: 0 | Self Similarit

  0%|          | 5/48828 [00:00<1:22:25,  9.87it/s]

representation shape is: torch.Size([8, 256, 512])
representation shape is: torch.Size([8, 256, 512])
representation shape is: torch.Size([8, 256, 512])


  0%|          | 7/48828 [00:00<1:19:06, 10.29it/s]

...

 18%|█▊        | 8864/48828 [13:39<1:02:03, 10.73it/s]

representation shape is: torch.Size([8, 256, 512])
representation shape is: torch.Size([8, 256, 512])
representation shape is: torch.Size([8, 256, 512])


 18%|█▊        | 8868/48828 [13:40<1:01:26, 10.84it/s]

representation shape is: torch.Size([8, 256, 512])
representation shape is: torch.Size([8, 256, 512])
representation shape is: torch.Size([8, 256, 512])


 18%|█▊        | 8870/48828 [13:40<1:01:16, 10.87it/s]

In [ ]:
model_save_name = cfg.model_name.split("/")[-1]
save_name = f"{model_save_name}_sp{cfg.sparsity}_r{cfg.ratio}_{tensor_names[0]}"  # trim year

# Make directory traiend_models if it doesn't exist
import os
if not os.path.exists("trained_models"):
    os.makedirs("trained_models")
# Save model
torch.save(autoencoder, f"trained_models/{save_name}.pt")

In [ ]:
wandb.finish()